<a href="https://colab.research.google.com/github/al2501/fooddesert/blob/main/Health_section.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creating Base Map and EFDI Score Choropleth

In [ ]:
#Import libraries and setup environment
import folium

!pip install pyproj 
from pyproj import crs

!pip install geopandas
import geopandas as gpd
from geopandas import GeoDataFrame

import matplotlib.pyplot as plt

import pandas as pd 

import branca.colormap as cm

from shapely.geometry import Point

     |████████████████████████████████| 6.3 MB 4.7 MB/s 
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 15.4 MB 39.7 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import cleaned and merged geofood dataset from initial choropleth mapping
data_path = "/content/drive/MyDrive/QM data/labels/geofood_final.shp"
geofood = gpd.read_file (data_path)
geofood.head()

,LAD20CD,Score,LAD20NM,geometry
0,E06000001,29.275,Hartlepool,"MULTIPOLYGON (((447213.899 537036.104, 447228...."
1,E06000002,28.090,Middlesbrough,"MULTIPOLYGON (((448609.900 521982.600, 448616...."
2,E06000003,26.601,Redcar and Cleveland,"MULTIPOLYGON (((455932.335 527880.697, 455919...."
3,E06000004,21.535,Stockton-on-Tees,"MULTIPOLYGON (((444157.002 527956.304, 444165...."
4,E06000005,23.984,Darlington,"POLYGON ((423496.602 524724.299, 423497.204 52..."


In [ ]:
geofood['geoid'] = geofood.index.astype(str)

In [ ]:
#Check for null values and clean
geofood.isna()

,LAD20CD,Score,LAD20NM,geometry,geoid
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
309,False,False,False,False,False
310,False,False,False,False,False
311,False,False,False,False,False
312,False,False,False,False,False


In [ ]:
geofood.dropna()

,LAD20CD,Score,LAD20NM,geometry,geoid
0,E06000001,29.275,Hartlepool,"MULTIPOLYGON (((447213.899 537036.104, 447228....",0
1,E06000002,28.090,Middlesbrough,"MULTIPOLYGON (((448609.900 521982.600, 448616....",1
2,E06000003,26.601,Redcar and Cleveland,"MULTIPOLYGON (((455932.335 527880.697, 455919....",2
3,E06000004,21.535,Stockton-on-Tees,"MULTIPOLYGON (((444157.002 527956.304, 444165....",3
4,E06000005,23.984,Darlington,"POLYGON ((423496.602 524724.299, 423497.204 52...",4
...,...,...,...,...,...
309,E09000029,13.012,Sutton,"POLYGON ((527085.942 167617.691, 527087.728 16...",309
310,E09000030,15.403,Tower Hamlets,"POLYGON ((536776.386 184446.881, 536777.600 18...",310
311,E09000031,15.264,Waltham Forest,"POLYGON ((537921.098 196048.201, 537947.499 19...",311
312,E09000032,10.026,Wandsworth,"POLYGON ((530059.620 177871.478, 530066.940 17...",312


In [ ]:
#Turn geofood into a GeoDataFrame
geofood = GeoDataFrame(geofood)

In [ ]:
type(geofood.crs)

NoneType

In [ ]:
#Creating a simple interactive web-map
mymap = folium.Map(location=[52.6, 0.15], zoom_start=6,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False) .add_to(mymap)
mymap

In [ ]:
from pyproj import CRS
geofood.crs = 'epsg:27700'
target_crs = {'datum':'OSGB36', 'no_defs':True, 'proj':'merc'}
projected_geofood= geofood.to_crs(crs=target_crs)

In [ ]:
# Importing and cleaning CHD dataset
data_path = "/content/drive/MyDrive/QM data/data health/chd.csv"
col_names = ["Code", "Label","Emergency Hospital Admissions for Coronary Heart Disease"]
chd = pd.read_csv(data_path, names=col_names, sep = ';')
chd = chd.drop([0,1,2])
chd.drop(columns=["Label"],inplace=True)
chd['Emergency Hospital Admissions for Coronary Heart Disease'] = pd.to_numeric(chd['Emergency Hospital Admissions for Coronary Heart Disease'], errors='coerce').fillna(0).astype(int)


chd.head()

,Code,Emergency Hospital Admissions for Coronary Heart Disease
3,E06000001,161
4,E06000002,120
5,E06000003,93
6,E06000004,151
7,E06000005,118


In [ ]:
# Importing and cleaning Cancer dataset
data_path = "/content/drive/MyDrive/QM data/data health/cancer.csv"
col_names = ["Code", "Label","Incidence of All Cancer"]
cancer = pd.read_csv(data_path, names=col_names, sep = ';')
cancer = cancer.drop([0,1,2])
cancer.drop(columns=["Label"],inplace=True)
cancer["Incidence of All Cancer"] = pd.to_numeric(cancer["Incidence of All Cancer"], errors='coerce').fillna(0).astype(int)
cancer.head()

,Code,Incidence of All Cancer
3,E06000001,108
4,E06000002,112
5,E06000003,107
6,E06000004,105
7,E06000005,103


In [ ]:
# Importing and cleaning Obesity dataset

data_path = "/content/drive/MyDrive/QM data/data health/obesity.csv"
col_names = ["Code", "Label","Estimated Prevalence of Obesity"]
obesity = pd.read_csv(data_path, names=col_names, sep = ';')
obesity = obesity.drop([0,1,2])
obesity["Estimated Prevalence of Obesity"] = pd.to_numeric(obesity["Estimated Prevalence of Obesity"], errors='coerce').fillna(0).astype(int)
obesity.drop(columns=["Label"],inplace=True)

obesity.head()

,Code,Estimated Prevalence of Obesity
3,E06000001,1
4,E06000002,1
5,E06000003,1
6,E06000004,1
7,E06000005,3


In [ ]:
#Merging health datasets together
merge1= chd.merge(cancer,right_on="Code",left_on="Code")
merge2= obesity.merge(merge1,right_on="Code",left_on="Code")
merge2.head()

,Code,Estimated Prevalence of Obesity,Emergency Hospital Admissions for Coronary Heart Disease,Incidence of All Cancer
0,E06000001,1,161,108
1,E06000002,1,120,112
2,E06000003,1,93,107
3,E06000004,1,151,105
4,E06000005,3,118,103


In [ ]:
#Merging health dataset with geofood geometry
merge2 = merge2.rename(columns={"Code":"LAD20CD"})
geofoodandhealth = merge2.merge(geofood,right_on="LAD20CD",left_on="LAD20CD")
geofoodandhealth.head()

,LAD20CD,Estimated Prevalence of Obesity,Emergency Hospital Admissions for Coronary Heart Disease,Incidence of All Cancer,Score,LAD20NM,geometry,geoid
0,E06000001,1,161,108,29.275,Hartlepool,"MULTIPOLYGON (((447213.899 537036.104, 447228....",0
1,E06000002,1,120,112,28.090,Middlesbrough,"MULTIPOLYGON (((448609.900 521982.600, 448616....",1
2,E06000003,1,93,107,26.601,Redcar and Cleveland,"MULTIPOLYGON (((455932.335 527880.697, 455919....",2
3,E06000004,1,151,105,21.535,Stockton-on-Tees,"MULTIPOLYGON (((444157.002 527956.304, 444165....",3
4,E06000005,3,118,103,23.984,Darlington,"POLYGON ((423496.602 524724.299, 423497.204 52...",4


In [ ]:
geofoodandhealth.dropna()

,LAD20CD,Estimated Prevalence of Obesity,Emergency Hospital Admissions for Coronary Heart Disease,Incidence of All Cancer,Score,LAD20NM,geometry,geoid
0,E06000001,1,161,108,29.275,Hartlepool,"MULTIPOLYGON (((447213.899 537036.104, 447228....",0
1,E06000002,1,120,112,28.090,Middlesbrough,"MULTIPOLYGON (((448609.900 521982.600, 448616....",1
2,E06000003,1,93,107,26.601,Redcar and Cleveland,"MULTIPOLYGON (((455932.335 527880.697, 455919....",2
3,E06000004,1,151,105,21.535,Stockton-on-Tees,"MULTIPOLYGON (((444157.002 527956.304, 444165....",3
4,E06000005,3,118,103,23.984,Darlington,"POLYGON ((423496.602 524724.299, 423497.204 52...",4
...,...,...,...,...,...,...,...,...
302,E09000029,4,89,100,13.012,Sutton,"POLYGON ((527085.942 167617.691, 527087.728 16...",309
303,E09000030,5,105,89,15.403,Tower Hamlets,"POLYGON ((536776.386 184446.881, 536777.600 18...",310
304,E09000031,5,100,96,15.264,Waltham Forest,"POLYGON ((537921.098 196048.201, 537947.499 19...",311
305,E09000032,5,82,100,10.026,Wandsworth,"POLYGON ((530059.620 177871.478, 530066.940 17...",312


In [ ]:
geofoodandhealth['geoid'] = geofoodandhealth.index.astype(str)

In [ ]:
geofoodandhealth['geometry'].head()

0    MULTIPOLYGON (((447213.899 537036.104, 447228....
1    MULTIPOLYGON (((448609.900 521982.600, 448616....
2    MULTIPOLYGON (((455932.335 527880.697, 455919....
3    MULTIPOLYGON (((444157.002 527956.304, 444165....
4    POLYGON ((423496.602 524724.299, 423497.204 52...
Name: geometry, dtype: geometry

In [ ]:
#Turn geofood into a GeoDataFrame
geofoodandhealth = GeoDataFrame(geofoodandhealth)

In [ ]:
from pyproj import CRS
geofoodandhealth.crs = 'epsg:27700'
target_crs = {'datum':'OSGB36', 'no_defs':True, 'proj':'merc'}
projected_geofoodandhealth= geofoodandhealth.to_crs(crs=target_crs)

In [ ]:
geofoodandhealth['Estimated Prevalence of Obesity'].max()

5

In [ ]:
#creating the first layer map 
bins = [0, 1, 2, 3, 4, 5]
a = folium.Choropleth(
    geo_data=projected_geofoodandhealth,
    data=geofoodandhealth,
    columns=['geoid', 'Estimated Prevalence of Obesity'],
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Estimated Prevalence of Obesity from 0 to 5',
    bins=[float(x) for x in bins],
    reset=True
).add_to(mymap)

mymap
mymap.save('Obesity3.html')

In [ ]:
geofoodandhealth['Incidence of All Cancer'].min()


78

In [ ]:
geofoodandhealth['Incidence of All Cancer'].max()


116

In [ ]:
geofoodandhealth['Incidence of All Cancer'].mean()

99.04885993485343

In [ ]:
bins = [0, 50, 100, 150]
b = folium.Choropleth(
    geo_data=projected_geofoodandhealth,
    data=geofoodandhealth,
    columns=['geoid', 'Incidence of All Cancer'],
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Incidence of All Cancer',
    bins=[float(x) for x in bins],
    reset=True
).add_to(mymap)

mymap
mymap.save('Cancer.html')

###MAP 3

In [ ]:
geofoodandhealth['Emergency Hospital Admissions for Coronary Heart Disease'].min()

52

In [ ]:
geofoodandhealth['Emergency Hospital Admissions for Coronary Heart Disease'].max()

192

In [ ]:
geofoodandhealth['Emergency Hospital Admissions for Coronary Heart Disease'].mean()

96.35179153094462

In [ ]:
bins = [0, 50, 100, 200]
c = folium.Choropleth(
    geo_data=projected_geofoodandhealth,
    data=geofoodandhealth,
    columns=['geoid', 'Emergency Hospital Admissions for Coronary Heart Disease'],
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Emergency Hospital Admissions for Coronary Heart Disease',
    bins=[float(x) for x in bins],
    reset=True
).add_to(mymap)

mymap
mymap.save('Heart.html')